In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import numpy as np
import pandas as pd
from PIL import Image
import random 
import matplotlib.pyplot as plt
import urllib
import cv2
import threading
import h5py
import json

In [2]:

with open('./RandomImages.json') as json_data:
    randomimages = json.load(json_data)['RandomImages']
 

downloadedimagespath = '/media/capptu/e6ac2520-92d9-4158-85f2-ab719a6db66d/AROD/DownloadAROD/ARODIMAGES/'

datasetsizes = {'Images':200}
datasets = ['Images']
notFoundPhoto = cv2.resize(cv2.imread('./photo_unavailable.png'),(224,224))

In [3]:
import multiprocessing
from multiprocessing import Pool,Manager
import time 

processesNumber = 200
successfullyDownloadedImages = []


def downloadImage (randomimage,index,images_dict,notFoundPhoto):
    for i in range(3):
        try:
            resp = urllib.urlopen(randomimage[0])
            image = np.asarray(bytearray(resp.read()), dtype="uint8")
            image = cv2.imdecode(image, cv2.IMREAD_COLOR)
            image = cv2.resize(image, (224,224),interpolation = cv2.INTER_CUBIC)           
            error = np.sum((image - notFoundPhoto))/(224*224*3)
            
            if (error < 5):
                image = np.zeros((10,10))
            break
        except:
            image = np.zeros((10,10))
            
    images_dict[index] = [image,randomimage]


    
    
def GetImageBatch(imageIndex, randomimages, notFoundPhoto):
    t1=time.time()
    manager = multiprocessing.Manager()
    images_dict = manager.dict()
    jobs = []
    for index in range(imageIndex,imageIndex+processesNumber):
        p = multiprocessing.Process(target=downloadImage,
                                    args=(randomimages[index],index,
                                    images_dict,notFoundPhoto))
        jobs.append(p)
        p.start()
        
    for proc in jobs:
        proc.join()
    #print return_dict.values()  
    image_values = images_dict.values()
    output = []
    for j in range (0,processesNumber):
        if np.shape(image_values[j]) != (10,10):
            output.append(image_values[j])
    t2=time.time()
    print "Finished batch downloading in: "+ str(t2-t1) +' seconds'
    return output

def appendZeros(num):
    maxdigits = 6
    digits = len (num)
    diff = maxdigits - digits
    return "0"*diff + num
    
    


In [4]:
imageIndex = 0
datasetIndex = 0
dataset = datasets[0]
#lset = g[dataset]
while datasetIndex < datasetsizes[dataset]:
    t1=time.time()        
    output = GetImageBatch(imageIndex,randomimages,notFoundPhoto)
    print 'output length' + str(len(output))
    for item in range(len(output)):
        if (datasetIndex < datasetsizes[dataset] )  and (np.shape(output[item][0]) ==(224,224,3)):    
            #lset[datasetIndex] = np.array([output[item][1][1],output[item][1][2],output[item][1][3]])
            cv2.imwrite('./ARODIMAGES/'+appendZeros(str(output[item][1][3])) +','+ str(output[item][1][1]) +','+ str(output[item][1][2])  + '.jpg',output[item][0])    
            datasetIndex = datasetIndex + 1
            if datasetIndex % 50 ==0:
                print 'ds  ' + str(datasetIndex) 
        else:
            print 'out of index'
    imageIndex = imageIndex + processesNumber 
    print 'datasetIndex ' + str(datasetIndex) 
    print 'imageIndex   ' + str(imageIndex) 
    
    print 'DatasetIndex' + str(datasetIndex)
    t2= time.time()
    print 'Batch elapsed time' +str(t2 - t1)+'s'

Finished batch downloading in: 39.987404108 seconds
output length200
ds  50
out of index
out of index
ds  100
out of index
ds  150
out of index
out of index
datasetIndex 195
imageIndex   200
DatasetIndex195
Batch elapsed time40.1122820377s


KeyboardInterrupt: 

Process Process-331:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-35c88c9dab71>", line 24, in downloadImage
    images_dict[index] = [image,randomimage]
  File "<string>", line 2, in __setitem__
  File "/usr/lib/python2.7/multiprocessing/managers.py", line 755, in _callmethod
    self._connect()
  File "/usr/lib/python2.7/multiprocessing/managers.py", line 742, in _connect
    conn = self._Client(self._token.address, authkey=self._authkey)
  File "/usr/lib/python2.7/multiprocessing/connection.py", line 169, in Client
    c = SocketClient(address)
  File "/usr/lib/python2.7/multiprocessing/connection.py", line 308, in SocketClient
    s.connect(address)
  File "/usr/lib/python2.7/socket.py", line 228, in meth
    return getattr(self._sock,name)(*args)
error: [Er

In [ ]:
# f = pd.read_csv('./list.txt',delimiter=';')
# imagesInfo = f.values
# randomValues = np.random.randint(len(imagesInfo), size=800000)
# randomimages ={'RandomImages':[imagesInfo[randomValues[i]].tolist()+[i] for i in range(len(randomValues))]}
# import json
# with open('RandomImages.json', 'w') as fp:
#     json.dump(randomimages, fp)

In [ ]:
# resp = urllib.urlopen('https://farm1.staticflickr.com/207/467756448_5d6038ab46_o.jpg')
# image = np.asarray(bytearray(resp.read()), dtype="uint8")
# image = cv2.imdecode(image, cv2.IMREAD_COLOR)
# image = cv2.resize(image, (224,224),interpolation = cv2.INTER_CUBIC)
# cv2.imwrite('color_img.jpg', image)